In [1]:
import time
from decimal import Decimal
from hypothesis import given, strategies as st


# Motivation

1. Verify correctness
2. Verify correctness after modifications (regression testing)
3. Example-based, how to use documentation
4. Provoke good design: SOLID, flexible, modular

<img src="https://www.worldofagile.com/wp-content/uploads/2018/03/Agile-Test-Pyramid.png">

In [2]:
# Let's say we want to implement frange

In [10]:
def frange(start, stop=None, step=1):
    if stop is None:
        stop = start
        start = 0
#     if step == 0:
#         raise ValueError('0 cant be used as step')

    start = Decimal(str(start))
    stop = Decimal(str(stop))
    step = Decimal(str(step))
    result = start

    while (step > 0 and result < stop) or (step < 0 and result > stop):
        yield result
        result += step


In [12]:
# How to verify correctness?

for i in frange(0.1, 5, 0.1):
    print(i)

0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0
1.1
1.2
1.3
1.4
1.5
1.6
1.7
1.8
1.9
2.0
2.1
2.2
2.3
2.4
2.5
2.6
2.7
2.8
2.9
3.0
3.1
3.2
3.3
3.4
3.5
3.6
3.7
3.8
3.9
4.0
4.1
4.2
4.3
4.4
4.5
4.6
4.7
4.8
4.9


In [15]:
# Primitive tests with asserts

assert list(frange(5)) == [0, 1, 2, 3, 4]
assert list(frange(2, 5)) == [2, 3, 4]
assert list(frange(2, 10, 2)) == [2, 4, 6, 8]
assert list(frange(10, 2, -2)) == [10, 8, 6, 4]
assert list(frange(2, 5.5, 1.5)) == [2, 3.5, 5]
assert list(frange(1, 5)) == [1, 2, 3, 4]
assert list(frange(0, 5)) == [0, 1, 2, 3, 4]
assert list(frange(1, 5, 0)) == []
assert list(frange(0, 0)) == []
assert list(frange(100, 0)) == []

print('SUCCESS')

SUCCESS


In [16]:
# Verbose error messages can be passed as a second param

assert list(frange(5)) == [0, 1, 2, 3, 6], f'{list(frange(5))} != [0, 1, 2, 3, 4, 6]'


# Is lazy evaluated on error, so inexpensive
assert list(frange(5)) == [0, 1, 2, 3, 4], [0]*10000000000000

AssertionError: [Decimal('0'), Decimal('1'), Decimal('2'), Decimal('3'), Decimal('4')] != [0, 1, 2, 3, 4, 6]

# Hypothesys

Good for computational test with ethalone to compare: (de)serializers, (de)encoders, sorting, etc

pip install hypothesis

https://hypothesis.readthedocs.io/en/latest/quickstart.html

In [17]:
@given(
    st.integers(min_value=-1000, max_value=1000),
    st.integers(min_value=-1000, max_value=1000),
    st.integers(min_value=-10, max_value=10).filter(lambda x: x != 0)
)
def test_frange(a, b, c):
    print(a, b, c)
    assert list(frange(a, b, c)) == list(range(a, b, c))

test_frange()

0 0 1
-532 226 -2
-370 339 -4
78 308 6
-473 -622 -8
522 779 -8
-259 796 -5
156 805 -4
-890 817 2
274 482 -4
-517 462 1
-517 -517 1
-517 -517 1
794 568 4
794 794 4
-437 946 -1
-437 -437 -1
-588 270 9
-588 588 9
-588 -588 9
-683 3 -7
-683 -683 -7
-158 -234 4
-158 -158 4
-297 740 7
-297 -297 7
-710 460 3
460 460 3
-380 -199 5
-380 -380 5
627 -237 -10
627 627 -10
605 950 -4
950 950 -4
350 590 -10
3 -590 -10
-590 -590 -10
203 -100 5
-203 -100 5
-100 -100 5
-101 -863 -3
-101 -101 -3
421 14 5
14 14 5
-564 -199 1
-564 -564 1
-564 -564 1
-576 -665 6
-665 -665 6
-491 -269 -3
-491 -491 -3
-627 8 -3
-8 8 -3
8 8 -3
-248 363 5
248 363 5
248 248 5
-589 -684 -3
-589 -684 -3
-589 -589 -3
-755 746 6
746 746 6
483 150 1
150 150 1
-444 237 1
-237 237 1
-237 -237 1
-593 -774 -3
-593 -593 -3
876 241 -10
241 241 -10
887 782 -7
887 887 -7
989 959 6
989 989 6
-22 -644 -5
-22 -22 -5
-22 -22 -5
490 81 5
81 81 5
-403 -136 3
-136 -136 3
-136 -136 3
-420 -245 4
-420 -420 4
-420 -420 4
422 -896 -9
422 422 -9
413 852

In [19]:
def partition(array, begin, end):
    pivot = begin
    for i in range(begin+1, end+1):
        if array[i] <= array[begin]:
            pivot += 1
            array[i], array[pivot] = array[pivot], array[i]
    array[pivot], array[begin] = array[begin], array[pivot]
    return pivot



def quicksort(array, begin=0, end=None):
    if end is None:
        end = len(array) - 1
    def _quicksort(array, begin, end):
        if begin >= end:
            return
        pivot = partition(array, begin, end)
        _quicksort(array, begin, pivot-1)
        _quicksort(array, pivot+1, end)
    return _quicksort(array, begin, end)

In [21]:
@given(st.lists(st.integers(), max_size=100))
def test_qsort(lst):
    print(lst)
    expected = sorted(lst)
    quicksort(lst)
    assert lst == expected, (lst, expected)
    
test_qsort()

[]
[0]
[0]
[9441, 16288]
[0]
[0]
[0]
[0]
[0]
[0]
[]
[12882, 9119, -1783792645314895563, -17762]
[-32707, 56, 9757]
[9757, 56, 9757]
[91]
[30799, -24, -21207, -15755, -30512]
[30799, -24, -21207, -15755, -30512]
[-14213, -94]
[-14213]
[18490, -20992]
[72, -4660092927399097764854112996021681622, 29630, 23665, -14353, 74, 4209, -6534, -26720, -1182, 3594, 89, 5129351824778240849]
[72, -4660092927399097764854112996021681622, 29630, 23665, -14353, 74, 4209, -6534, -26720, -1182, 3594, 89, 5129351824778240849]
[67, -834, 19207, -7322, -150236637648260360454424245055849451233, 78, 28, -38, 71, 1798033064, -144, 11371]
[-19442, -421194527, 13141, -407152484768392093]
[-19442, 10033899, 13141, -407152484768392093]
[-203, 10033899, 13141, -407152484768392093]
[57282908695593113, 58720435, -25264261, 4370, 7715, 24444, -14448, -834462625, 32359, -15096, -23157, -14308, -11358, -12080]
[23336, -6036]
[83652941733100868067998201516639094581, -48, -15, 47, 120221810532666628152087264838303417222, 57

SyntaxError: invalid syntax (Temp/ipykernel_16084/809897477.py, line 4)

# Module unittests

More versatile tool and flexible than plain asserts

In [24]:
import unittest

# 1. normal data
# 2. corner values (1, -1)
# 3. corner values + 1
# 4. wrong types
# 5. exceptions
# 6. empty values


class TestFrange(unittest.TestCase):

    def test_frange_single_param(self):
#         assert (list(frange(7)) == [0, 1, 2, 3, 4])
        self.assertEqual(list(frange(5)), [0, 1, 2, 3, 4, 5])
    
    def test_frange_2_params(self):
        self.assertEqual(list(frange(2, 5)), [2, 3, 4])
        
    def test_frange_2_params_empty_result(self):
        self.assertEqual(list(frange(5, 2)), [])

    def test_frange_custom_step(self):
        self.assertEqual(list(frange(2, 10, 2)), [2, 4, 6, 8])

    def test_frange_reverse_order(self):
        self.assertEqual(list(frange(10, 2, -2)), [10, 8, 6, 4])

unittest.main(argv=[''], verbosity=2, exit=False)

test_truncate_operation (__main__.TestFileOperations) ... ERROR
test_write_on_closed_file (__main__.TestFileOperations) ... ERROR
test_write_operation (__main__.TestFileOperations) ... ERROR
test_frange_2_params (__main__.TestFrange) ... ok
test_frange_2_params_empty_result (__main__.TestFrange) ... ok
test_frange_custom_step (__main__.TestFrange) ... ok
test_frange_reverse_order (__main__.TestFrange) ... ok
test_frange_single_param (__main__.TestFrange) ... FAIL

ERROR: test_truncate_operation (__main__.TestFileOperations)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\mikol\AppData\Local\Temp\ipykernel_30304\2613248743.py", line 5, in setUp
    self._file = open('/tmp/file.txt', 'w+')
FileNotFoundError: [Errno 2] No such file or directory: '/tmp/file.txt'

ERROR: test_write_on_closed_file (__main__.TestFileOperations)
----------------------------------------------------------------------
Traceback (most recen

setUp
setUp
setUp


#### Run:

python -m unittests test_frange.py

unittest.main()

In [ ]:
# Names are important: class Test*, test_* for methods

In [7]:
class TestFileOperations(unittest.TestCase):

    def setUp(self):
        print('setUp')
        self._file = open('/tmp/file.txt', 'w+')

    def tearDown(self):
        print('tearDown')
        self._file.close()

    def test_write_operation(self):
        data = 'test'
        self._file.write(data)
        self._file.seek(0)
        data_read = self._file.read()
        self.assertEqual(data_read, data)

    def test_truncate_operation(self):
        data = 'test'
        self._file.write(data)
        self._file.truncate(0)
        data_read = self._file.read()
        self.assertEqual(data_read, '')

    def test_write_on_closed_file(self):
        data = 'test'
        self._file.write(data)
        self._file.close()
        with self.assertRaises(ValueError) as ex:
            self._file.write(data)
        self.assertIsInstance(ex.exception, ValueError)

    ...

# PyTest

unittests vs pytest:
1. Less verbose code, more verbose error description
2. Pythonic (PEP-8), doesn't enforce classes
3. Compatible with unittests (recognize TestCase classes and execute them)

pip install pytest

python -m pytest tested_file.py

In [25]:
def test_passing():
    assert (1, 2, 3) == (1, 2, 3)
    
# test is ready :)

In [9]:
import pytest
import ipytest

@pytest.fixture
def file():
    file = open('/tmp/file.txt', 'w+')
    yield file
    file.close()

def test_write_operation(file):
    data = 'test'
    file.write(data)
    file.seek(0)
    data_read = file.read()
    assert data_read == data

def test_truncate_operation(file):
    data = 'test'
    file.write(data)
    file.truncate(0)
    data_read = file.read()
    assert data_read == ''

def test_write_on_closed_file(file):
    data = 'test'
    file.write(data)
    file.close()
    with pytest.raises(ValueError) as ex:
        file.write(data)
    assert isinstance(ex.value, ValueError)

ipytest.run()

======================================= test session starts =======================================
platform win32 -- Python 3.10.9, pytest-7.1.2, pluggy-1.0.0
rootdir: D:\Hillel\python_materials
plugins: anyio-3.5.0, hypothesis-6.45.1
collected 13 items

tmp9ot6npiy.py .....FFFF.EEE                                                                 [100%]

============================================= ERRORS ==============================================
_____________________________ ERROR at setup of test_write_operation ______________________________

    @pytest.fixture
    def file():
>       file = open('/tmp/file.txt', 'w+')
E       FileNotFoundError: [Errno 2] No such file or directory: '/tmp/file.txt'

C:\Users\mikol\AppData\Local\Temp\ipykernel_30304\2737270264.py:6: FileNotFoundError
____________________________ ERROR at setup of test_truncate_operation ____________________________

    @pytest.fixture
    def file():
>       file = open('/tmp/file.txt', 'w+')
E       FileNotFo

<ExitCode.TESTS_FAILED: 1>

ERROR: usage: ipykernel_launcher.py [options] [file_or_dir] [file_or_dir] [...]
ipykernel_launcher.py: error: unrecognized arguments: -f
  inifile: None
  rootdir: D:\Hillel\python_materials



<ExitCode.USAGE_ERROR: 4>

# Best practices

1. Cover corner cases: 0, 1, -1, len(lst)-1, len(lst)+1
   - Test symmetry behaviour is also often good idea: a + b == b + a, decode/encode, load/dump, etc
2. Cover all groups (equivalence class) of inputs: positive, negative, empty, large, small, ...
3. Don't repeate tests for the same class: it is redundant.
4. Tests should be isolated from each other (no order dependency)
   - unittest might run tests in arbitrary order
5. Tests should be atomic (test 1 feature), small and fast
6. Tests should cover not only positive (when code works as expected), but negative cases as well (when code doesn't work when it is not supposed)
   - test that errors are returned and exceptions are thrown
7. External dependencies can be replaced with mocks.
   - We don't want to test system libraries or frameworks

In [33]:
# Bad examples

class TestFrange(unittest.TestCase):

    def test_frange_single_param(self):
        self.assertEqual(list(frange(5)), [0, 1, 2, 3, 4])
        self.assertEqual(list(frange(7)), [0, 1, 2, 3, 4, 5, 6])

    def test_frange_2_params(self):
        self.assertEqual(list(frange(2, 5)), [0, 1, 2, 3, 4, 5])
        self.assertEqual(list(frange(5, 2)), [])
    